In [ ]:
# default_exp data_provider

In [ ]:
#export
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import pandas as pd
from fastcore.foundation import patch

# Data Provider

> This module is responsible for processing raw TRR data for further analysis. ToDo: This code should be formatted as a class that gets the data folder path on init.

In [ ]:
#export
class DataProvider():
    def __init__(self, data_folder_path):
        self.data_folder_path = data_folder_path
        self.raw = os.path.join(data_folder_path, 'raw')
        self.external = os.path.join(data_folder_path, 'external')
        self.interim = os.path.join(data_folder_path, 'interim')
        self.processed = os.path.join(data_folder_path, 'processed')
        # Checking if folder paths exist
        assert os.path.isdir(self.external), "External data folder not found."
        assert os.path.isdir(self.raw), "Raw data folder not found."
        assert os.path.isdir(self.interim), "Interim data folder not found."
        assert os.path.isdir(self.processed), "Processed data folder not found."
        # Phone screening files
        self.phonescreening_data_path = os.path.join(self.raw, "phonescreening.csv")
        self.phone_codebook_path = os.path.join(self.external, "phone_codebook.html")
        # Basic assessment files
        self.ba_codebook_path = os.path.join(self.external, "ba_codebook.html")
        self.ba_data_path = os.path.join(self.raw, "ba.csv")
        self.b07_participants_path = os.path.join(self.external, "b7_participants.xlsx")
        # Movisense data
        self.mov_berlin_path = os.path.join(self.raw, "mov_data_b.csv")
        self.mov_dresden_path = os.path.join(self.raw, "mov_data_d.csv")
        self.mov_mannheim_path = os.path.join(self.raw, "mov_data_m.csv")
        self.mov_berlin_starting_dates_path = os.path.join(self.raw, "starting_dates_b.html")
        self.mov_dresden_starting_dates_path = os.path.join(self.raw, "starting_dates_d.html")
        self.mov_mannheim_starting_dates_path = os.path.join(self.raw, "starting_dates_m.html")
        self.alcohol_per_drink_path = os.path.join(self.external,'alcohol_per_drink.csv')


#export
def get_efficiently(func):
    """
    This decorator wraps around functions that get data and handles data storage.
    If the output from the function hasn't been stored yet, it stores it in "[path_to_interim]/[function_name_without_get].parquet"
    If the output from the function has been stored already, it loads the stored file instead of running the function (unless update is specified as True)
    """
    def w(*args, update = False, columns = None, path = None, **kw):
        _self = args[0] # Getting self to grab interim path from DataProvider
        var_name = func.__name__.replace('__get_','').replace('get_','')
        file_path = os.path.join(_self.interim, "%s.parquet"%var_name)
        if os.path.exists(file_path) and (update == False):
            result =  pd.read_parquet(file_path, columns = columns)
        else:
            print("Preparing %s"%var_name)
            result = func(_self)
            result.to_parquet(file_path)
        return result
    w.__wrapped__ = func # Specifying the wrapped function for inspection
    w.__doc__ = func.__doc__
    w.__name__ = func.__name__
    w.__annotations__ = {'cls':DataProvider, 'as_prop':False} # Adding parameters to make this work with @patch
    return w

In [ ]:
dp = DataProvider('/Users/hilmarzech/Projects/trr265/trr265/data/')

## Functions

In [ ]:
#export
@patch
def store_interim(self:DataProvider, df, filename):
    path = os.path.join(self.interim,"%s.parquet"%filename)
    df.to_parquet(path)

In [ ]:
#export
@patch
def load_interim(self:DataProvider, filename):
    return pd.read_parquet(os.path.join(self.interim,"%s.parquet"%filename))

In [ ]:
def storing_test(dp):
    df = pd.DataFrame({"test":[4]})
    # Storing the dataframe
    dp.store_interim(df, 'storing_test')
    # Reloading it and comparing to original
    return df.equals(dp.load_interim('storing_test'))

In [ ]:
assert storing_test(dp), "Data storage is failing."

### Getting phone screening data
> This functions get the phone screening data from redcap.  To run them, first go to redcap and download the phonescreening data to 'raw/phonescreening.csv' and the phonescreening codebook to 'external/phone_codebook.html'.  You also need the external/b7_participants.xlsx (if it's not in the cloned package, ask Hilmar for it.

In [ ]:
#export
@patch
@get_efficiently
def get_phone_codebook(self:DataProvider):
    tables = pd.read_html(open(self.phone_codebook_path,'r').read())
    df = tables[1]
    # Note that str.contains fills NaN values with nan, which can lead to strange results during filtering
    df = df[df.LabelHinweistext.str.contains('Fragebogen:',na=False)==False]
    df = df.set_index('#')
    # Parsing variable name
    df['variable'] = df["Variable / Feldname"].apply(lambda x: x.split(' ')[0])
    # Parsing condition under which variable is displayed
    df['condition'] = df["Variable / Feldname"].apply(lambda x: ' '.join(x.split(' ')[1:]).strip() if len(x.split(' '))>1 else '')
    df['condition'] = df.condition.apply(lambda x: x.replace('Zeige das Feld nur wenn:  ',''))
    # Parsing labels for numerical data
    df['labels'] = np.nan
    labels = tables[2:-1]
    try:
        labels = [dict(zip(l[0],l[1])) for l in labels]
    except:
        display(table)
    searchfor = ["radio","dropdown","yesno","checkbox"]
    with_table = df['Feld Attribute (Feld-Typ, Prüfung, Auswahlen, Verzweigungslogik, Berechnungen, usw.)'].str.contains('|'.join(searchfor))
    df.loc[with_table,'labels'] = labels
    df = df.astype(str)
    return df

In [ ]:
assert 'participant_id' in dp.get_phone_codebook().variable.values, "Failed to load phone_codebook.html"

> Initially, participants from b07 were stored in the same RedCap project as S01 participants.  This function attempts to disentangle the two

In [ ]:
#export 
@patch
def determine_phone_b07(self:DataProvider, df):
    # Some initial fixes
    df.loc[df.center=='d','screen_caller'] = df.loc[df.center=='d','screen_caller'].str.lower().str.strip().replace('leo','leonard visser').replace('sebastian mörcke','sebastian möricke').replace('jessica zimmerman','jessica zimmermann').replace('miriam-sophie petasch','miriam petasch').replace('dorothee','dorothee scheuermann')
    # Cleaning screener list
    dd_screeners = df[(df.center=='d')&(df.screen_caller.isna()==False)].screen_caller.unique()
    def clean_screeners(dd_screeners):
        dd_screeners = [y  for x in dd_screeners for y in x.split('+')]
        dd_screeners = [y  for x in dd_screeners for y in x.split(',')]
        dd_screeners = [y  for x in dd_screeners for y in x.split('und')]
        dd_screeners = [y.replace('(15.02.21)','')  for x in dd_screeners for y in x.split('/')]
        dd_screeners = [y.replace(')','').strip().lower()  for x in dd_screeners for y in x.split('(')]
        dd_screeners = sorted(list(set(dd_screeners)))
        return dd_screeners
    dd_screeners = clean_screeners(dd_screeners)

    b07_screeners = ['ann-kathrin stock','charlotte blum','josephine kirschgens','klara macht','borchardt','marta ledro','miriam petasch','mona hofmann','theo tester']
    s01_screeners = ['esther preuschhof', 'miriam schmitz', 'sebastian möricke', 'jessica zimmermann', 'leonard visser', 'anna-lena lünert', 'anne dörfler', 'dominic reichert', 'maike borchardt', 'dorothee scheuermann', 'paula böhlmann', 'alice','josephine kirschgens maxi stiller', 'maria schießl', '22.10.2021', 'sascha', '03.08.2021', 'mariana plumbohm', 'caroline neumer', '04.08.2021', 'maxi stiller', 'sacsha', '09.08.2021', 'ml', 'charlotte', 'charlotte heinze', 'shereen', 'test', "charlotte heinze", 'benedikt','"charlotte heinze"', 'shereen qedra', 'antonia hunnius', 'anna-lena', 'anna wildenauer', 'antonia', 'anni dittrich']
    known_dd_screeners = list(b07_screeners+s01_screeners)
    dd_screeners = df[(df.center=='d')&(df.screen_caller.isna()==False)].screen_caller.unique()
    # Checking if all Dresden phone screeners are accounted for
    assert df[(df.center=='d')&(df.screen_caller)].screen_caller.str.contains('|'.join(known_dd_screeners)).mean()==1, "Unknown Dresden phone screener: %s"%', '.join(set(clean_screeners(dd_screeners))-set(known_dd_screeners))
    # In general, if a screener from a project was involved, it was screened for that project
    df['screened_for_b07'] = (df.center=='d') & (df.screen_caller.str.contains('|'.join(b07_screeners)))
    df['screened_for_s01'] = (df.center!='d') | (df.screen_caller.str.contains('|'.join(s01_screeners)))

    # We also exclude participants screened for C02 in Berlin
    df.loc[(df.screen_purpose == 4) & (df.center=='b'), 'screened_for_s01'] = False

    # Additionally, we also set it to true if it was specifically set
    df.loc[df.screen_site_dd == 1, 'screened_for_s01'] = True
    df.loc[df.screen_site_dd == 3, 'screened_for_s01'] = True
    df.loc[df.screen_site_dd == 2, 'screened_for_b07'] = True
    df.loc[df.screen_site_dd == 3, 'screened_for_b07'] = True
    return df

In [ ]:
#export
@patch
def check_participant_id(self:DataProvider,x):
    '''This function checks whether a participant ID is numerical and lower than 20000.'''
    if str(x) == x:
        if x.isnumeric():
            x = float(x)
        else:
            return False
    if x > 20000:
        return False
    return True

In [ ]:
#export
@patch
def test_check_participant_id(self:DataProvider):
    failed = dp.check_participant_id('test10') == False # Example of a bad participant ID
    passed = dp.check_participant_id('100') == True # Example of a good participant ID
    return failed and passed

In [ ]:
assert dp.test_check_participant_id(), "Participant ID check is not working"

In [ ]:
#export
@patch
def set_dtypes(self:DataProvider, data, codebook):
    def number_or_nan(x):
        try:
            float(x)
            return x
        except:
            return np.nan
    '''This function automatically adjust data types of redcap data based on the redcap codebooks'''
    # Parsing type
    codebook['type'] = codebook["Feld Attribute (Feld-Typ, Prüfung, Auswahlen, Verzweigungslogik, Berechnungen, usw.)"].apply(lambda x: x.split(',')[0])
    # Descriptives (not in data)
    desc_columns = list(codebook[codebook.type.str.contains('descriptive')].variable)
    # Datetime
    dt_columns = codebook[(codebook.type.isin(['text (datetime_dmy)','text (date_dmy)']))].variable
    dt_columns = list(set(data.columns).intersection(dt_columns))
    # Numerical
    num_columns = []
    num_columns += list(codebook[codebook.type.str.contains('calc')].variable)
    num_columns += list(codebook[codebook.type.str.contains('checkbox')].variable)
    num_columns += list(codebook[codebook.type.str.contains('radio')].variable)
    num_columns += list(codebook[codebook.type.str.contains('text \(number')].variable)
    num_columns += list(codebook[codebook.type.str.contains('yesno')].variable)
    num_columns += list(codebook[codebook.type.str.contains('dropdown')].variable)
    num_columns += list(codebook[codebook.type.str.contains('slider')].variable)
    num_columns = list(set(data.columns).intersection(num_columns))
    # Text
    text_columns = []
    text_columns += list(codebook[(codebook.type.str.contains('text')) & (~codebook.type.str.contains('date_dmy|datetime_dmy'))].variable)
    text_columns += list(codebook[(codebook.type.str.contains('notes'))].variable)
    text_columns += list(codebook[(codebook.type.str.contains('file'))].variable)
    text_columns = list(set(data.columns).intersection(text_columns))
    assert len(set(num_columns).intersection(set(dt_columns)))==0, set(num_columns).intersection(set(dt_columns))
    assert len(set(text_columns).intersection(set(dt_columns)))==0, set(text_columns).intersection(set(dt_columns))



    for c in num_columns:
        data[c].replace("A 'MySQL server has gone away' error was detected.  It is possible that there was an actual database issue, but it is more likely that REDCap detected this request as a duplicate and killed it.", np.nan, inplace = True)
        try:
            data[c] = data[c].astype(float)
        except:
            data[c] = data[c].apply(number_or_nan).astype(float)
            print("Values with wrong dtype in %s"%c)
    data[text_columns] = data[text_columns].astype(str).replace('nan',np.nan)

    for c in dt_columns:
        data[c] = pd.to_datetime(data[c])
    return data

In [ ]:
#export
@patch
@get_efficiently
def get_phone_data(self:DataProvider):
    df = pd.read_csv(self.phonescreening_data_path,
                     na_values = ["A 'MySQL server has gone away' error was detected.  It is possible that there was an actual database issue, but it is more likely that REDCap detected this request as a duplicate and killed it."]
                    )
    remove = ['050571', '307493', '345678', '715736', 'Ihloff', 'test',
       'test002', 'test003', 'test004', 'test005', 'test01', 'test02',
       'test03', 'test0722', 'test1', 'test34', 'test999', 'test2020',
       'test20201', 'test345345', 'testt', 'test_10', 'test_11_26',
       'test_neu', 'xx956','050262', '050335', '050402', '050416', '051005', '294932', '891752080', '898922719', '898922899', '917702419', '01627712983', 'meow', 'test0022', 'test246', 'test5647', 'test22222', 'test41514', 'testtt', 'test_057', 'tets','898923271', 'test001', 'test006', 'test007', 'test008', 'test11', 'test_23_12', 'test_n','50744', 'test0001a', 'test004', 'test03', 'tets', 'test_neu', 'xx956','050262', '050335', '050402', '050416', '051005', '294932', '891752080', '898922719', '898922899', '917702419', '01627712983', 'meow', 'test0022', 'test246', 'test5647', 'test22222', 'test41514', 'testtt', 'test_057','050687', '5505538', '34567896', '123456789', '978770809', 'test0012', 'test0012a', 'test00123', 'test0013', 'test0016', 'test009', 'test010', 'test011', 'test012', 'test013', 'test22', 'test33', 'test123', 'test203', 'test227', 'test22223', 'testtestt', 'test_001', 'tets2']
    df = df[~df.participant_id.astype(str).isin(remove)]

    bad_ids = df[~df.participant_id.apply(self.check_participant_id)].participant_id.unique()
    assert len(bad_ids)==0, "Bad participant IDs (should be added to remove): %s"%', '.join(["'%s'"%b for b in bad_ids])
    self.get_phone_codebook()
    df = self.set_dtypes(df, self.get_phone_codebook())
    df['participant_id'] = df.participant_id.astype(int)
    df['center'] = df.screen_site.replace({1:'b',2:'d',3:'m'})
    df['screen_date'] = pd.to_datetime(df['screen_date'], errors = 'coerce')
    #display(df[df.screen_caller.isna()])
    df = self.determine_phone_b07(df)
    df['participant_id'] = df.participant_id.apply(lambda x: '%05d'%int(x))
    return df

In [ ]:
phone = dp.get_phone_data(update=True)

Preparing phone_data


In [ ]:
assert 'screen_caller' in dp.get_phone_data().columns, "Failed to load phone data."

### Getting baseline data
> Warning: This function does not work if the computers default encoding is not UTF-8 -> This should be adjusted in pd.readhtml...

In [ ]:
#export
@patch
@get_efficiently
def get_ba_codebook(self:DataProvider):
    tables = pd.read_html(open(self.ba_codebook_path,"r", encoding ='UTF-8').read())
    df = tables[1]
    # Note that str.contains fills NaN values with nan, which can lead to strange results during filtering
    df = df[df.LabelHinweistext.str.contains('Fragebogen:',na=False)==False]
    df = df.set_index('#')
    # Parsing variable name
    df['variable'] = df["Variable / Feldname"].apply(lambda x: x.split(' ')[0])
    # Parsing condition under which variable is displayed
    df['condition'] = df["Variable / Feldname"].apply(lambda x: ' '.join(x.split(' ')[1:]).strip() if len(x.split(' '))>1 else '')
    df['condition'] = df.condition.apply(lambda x: x.replace('Zeige das Feld nur wenn:  ',''))
    # Parsing labels for numerical data
    df['labels'] = np.nan
    labels = tables[2:-1]
    try:
        labels = [dict(zip(l[0],l[1])) for l in labels]
    except:
        display(table)
    searchfor = ["radio","dropdown","yesno","checkbox"]
    with_table = df['Feld Attribute (Feld-Typ, Prüfung, Auswahlen, Verzweigungslogik, Berechnungen, usw.)'].str.contains('|'.join(searchfor))
    df.loc[with_table,'labels'] = labels
    df = df.astype(str)
    return df

In [ ]:
assert 'variable' in dp.get_ba_codebook(), "Codebook did not load correctly"

> Warning: get_ba_data uses a different way to remove b07 participants.  This should be adjusted to use the same function as get_phone_data

In [ ]:
#export
@patch
@get_efficiently
def get_ba_data(self:DataProvider):
    '''This function reads in baseline data from redcap, filters out pilot data, and creates movisens IDs.'''
    df = pd.read_csv(self.ba_data_path)
    df['center'] = df.groupby('participant_id').bx_center.transform(lambda x: x.ffill().bfill())
    df['center'] = df.center.replace({1:'b',2:'d',3:'m'})
    # Creating new movisense IDs (adding center prefix to movisense IDs)
    for old_id in ['bx_movisens','bx_movisens_old','bx_movisens_old_2']:
        new_id = old_id.replace('bx_','').replace('movisens','mov_id')
        df[new_id] = df.groupby('participant_id')[old_id].transform(lambda x: x.ffill().bfill())
        df[new_id] = df.center + df[new_id].astype('str').str.strip('0').str.strip('.').apply(lambda x: x.zfill(3))
        df[new_id].fillna('nan',inplace = True)
        df.loc[df[new_id].str.contains('nan'),new_id] = np.nan
    # Removing test participants
    remove = ['050744', 'hdfghadgfh', 'LindaEngel', 'test', 'Test001', 'Test001a', 'test0011', 'test0012', 'test0013', 'test0014', 'test0015', 'test002', 'test00229', 'test007', 'test01', 'test012', 'test013', 'test1', 'test2', 'test4', 'test12', 'test999', 'test2021', 'test345345', 'testneu', 'testtest', 'test_0720', 'test_10', 'test_GA', 'Test_JH','test0016','891752080', 'pipingTest', 'test0001', 'test00012', 'test0012a', 'test0015a', 'test0017', 'test10', 'test20212', 'testJohn01', 'test_00213', 'test_00233', 'test_00271', 'test_003', 'test_004', 'test_11_26', 'Test_MS','898922899', 'tesst', 'test0002', 'test0908', 'test092384750398475', 'test43', 'test123', 'test1233', 'test3425', 'test123456', 'test1234567', 'testfu3', 'test_888', 'test_999', 'test_98375983745', 'Test_Übung','050335', 'test003', 'test02', 'test111', 'test1111', 'test1234','test0000', 'test_CH','50744', 'test0001a', 'test004', 'test03', 'tets','test009', 'test22', 'test912', 'test3003', 'test11111', 'test_MH','88888', 'test00001', 'test333', 'test_20220623']
    df = df[~df.participant_id.astype(str).isin(remove)]
    # Checking participant ids (to find new test participants)
    bad_ids = df[~df.participant_id.apply(self.check_participant_id)].participant_id.unique()
    assert len(bad_ids)==0, "Bad participant IDs (should be added to remove): %s"%', '.join(["'%s'"%b for b in bad_ids])
    # labeling B07 participant
    b07_pps = pd.read_excel(self.b07_participants_path)['Participant ID'].astype(str)
    df['is_b07'] = False
    df.loc[df.participant_id.isin(b07_pps),'is_b07'] = True
    # Setting dtypes based on codebook
    df = self.set_dtypes(df, self.get_ba_codebook())
    # Creating convenience variables
    df['is_female'] = df.screen_gender.replace({1:1,2:0,3:np.nan})
    # Filling in missings from baseline
    df['is_female'].fillna(df.bx_sozio_gender.replace({1:0,2:1,3:np.nan}), inplace = True)
    df['is_female'] = df.groupby('participant_id')['is_female'].transform(lambda x: x.ffill().bfill())
    df['is_female'] = df['is_female'].astype(float)
    df['participant_id'] = df.participant_id.apply(lambda x: '%05d'%int(x))
    return df

> ToDo: This should be defined as a "slow test"

According to the redcap data, there should be 581 participants.

In [ ]:
check_ba = dp.get_ba_data(update=True)[['participant_id','redcap_event_name','redcap_repeat_instrument','redcap_repeat_instance']].value_counts().mean() == 1
assert check_ba, "Redcap BA data was not properly formatted"

Preparing ba_data


In [ ]:
dp.get_ba_data().query('mov_id=="m101"')[['bx_sozio_gender','screen_gender','is_female']]

,bx_sozio_gender,screen_gender,is_female
10505,1.0,2.0,0.0
10506,NaN,NaN,0.0
10507,NaN,2.0,0.0
10508,NaN,2.0,0.0
10509,NaN,2.0,0.0
10510,NaN,NaN,0.0
10511,NaN,NaN,0.0
10512,NaN,NaN,0.0
10513,NaN,NaN,0.0
10514,NaN,NaN,0.0


In [ ]:
dp.get_ba_data()

,participant_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_survey_identifier,timestamp_prepques,study_id,bx_date,languages,bx_center,...,comment_normalization,mr_inclusion,comment_excl_reason_ua,fmri_nicepype_qa_report_complete,center,mov_id,mov_id_old,mov_id_old_2,is_b07,is_female
0,00053,erhebungszeitpunkt_arm_1,None,NaN,NaN,None,10101.0,2021-10-20,NaN,1.0,...,NaN,NaN,None,NaN,b,None,None,None,False,1.0
1,00053,mrterhebung_1_arm_1,None,NaN,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,None,None,None,False,1.0
2,00053,biomaterialien_arm_1,None,NaN,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,None,None,None,False,1.0
3,00053,erhebungszeitpunkt_arm_1,at_home_qsu,1.0,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,None,None,None,False,1.0
4,00053,erhebungszeitpunkt_arm_1,at_home_ftnd,1.0,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,None,None,None,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20270,12804,mrterhebung_1_arm_1,examiner_drugs,2.0,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,b372,None,None,False,1.0
20271,12804,mrterhebung_1_arm_1,neuerfb,1.0,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,b372,None,None,False,1.0
20272,12804,mrterhebung_1_arm_1,clinic_participant_panas,1.0,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,b372,None,None,False,1.0
20273,12804,mrterhebung_1_arm_1,clinic_examiner_mrt_examination,1.0,NaN,None,None,NaT,NaN,NaN,...,NaN,NaN,None,NaN,b,b372,None,None,False,1.0


In [ ]:
ba = dp.get_ba_data(update = True)

Preparing ba_data


In [ ]:
ba.bx_date.max()

Timestamp('2022-08-03 00:00:00')

In [ ]:
gba = pd.read_stata('/Users/hilmarzech/Projects/trr265/trr265/data/raw/TRR265_PID78_20220623/BA_S01_t0.dta',convert_categoricals=False)

In [ ]:
gba['date'] = pd.to_datetime(gba.t0_bx_date)

In [ ]:
gba_pps = gba.query("date<20220321").participant_id

In [ ]:
ba_pps = ba.participant_id

In [ ]:
ba_pps

0        00053
1        00053
2        00053
3        00053
4        00053
         ...  
20270    12804
20271    12804
20272    12804
20273    12804
20274    12814
Name: participant_id, Length: 20275, dtype: object

In [ ]:
set(gba_pps) - set(ba_pps)

set()

In [ ]:
test = pd.merge(left = ba.groupby('participant_id').first().reset_index(), how ='outer', right = gba, on = 'participant_id', indicator = True)

In [ ]:
test._merge.value_counts()

both          809
left_only     230
right_only      0
Name: _merge, dtype: int64

In [ ]:
test.query('_merge=="right_only"').date_y.min()

NaT

Only participants that were screened after 22.3.2022 are not in the ba dataframe. The ba dataframe includes participants that were not included in the study, whereas these are filtered in the gba dataframe.

ToDo: Are age and gender the same in both dataframes?

In [ ]:
test.set_index('participant_id').filter(like='gender')

,screen_gender,bx_sozio_gender,bx_sozio_genderother,t0_screen_gender,t0_bx_sozio_gender,t0_bx_sozio_genderother
participant_id,,,,,,
00053,1.0,2.0,None,1.0,2.0,
00067,1.0,2.0,None,1.0,2.0,
00075,1.0,2.0,None,1.0,2.0,
00078,2.0,NaN,None,NaN,NaN,NaN
00079,2.0,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...
12790,2.0,1.0,None,NaN,NaN,None
12791,1.0,2.0,None,NaN,NaN,None
12803,2.0,NaN,None,NaN,NaN,None


In [ ]:
ba[ba.participant_id.isin((set(ba_pps) - set(gba.participant_id)))].query('mov_id.isna()==False')

,participant_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,redcap_survey_identifier,timestamp_prepques,study_id,bx_date,languages,bx_center,...,comment_normalization,mr_inclusion,comment_excl_reason_ua,fmri_nicepype_qa_report_complete,center,mov_id,mov_id_old,mov_id_old_2,is_b07,is_female
602,00224,erhebungszeitpunkt_arm_1,NaN,NaN,NaN,2022-02-21 13:18,10101.0,2021-07-16,NaN,2.0,...,NaN,NaN,NaN,NaN,d,d815,NaN,NaN,False,1.0
603,00224,followup_1_arm_1,NaN,NaN,NaN,NaN,NaN,2021-07-30,NaN,NaN,...,NaN,NaN,NaN,NaN,d,d815,NaN,NaN,False,1.0
604,00224,followup_2_arm_1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,d,d815,NaN,NaN,False,1.0
605,00224,followup_3_arm_1,NaN,NaN,NaN,NaN,NaN,2022-02-02,NaN,NaN,...,NaN,NaN,NaN,NaN,d,d815,NaN,NaN,False,1.0
606,00224,dropout_arm_1,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,d,d815,NaN,NaN,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20269,12804,mrterhebung_1_arm_1,examiner_drugs,1.0,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,b,b372,NaN,NaN,False,1.0
20270,12804,mrterhebung_1_arm_1,examiner_drugs,2.0,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,b,b372,NaN,NaN,False,1.0
20271,12804,mrterhebung_1_arm_1,neuerfb,1.0,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,b,b372,NaN,NaN,False,1.0
20272,12804,mrterhebung_1_arm_1,clinic_participant_panas,1.0,NaN,NaN,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,b,b372,NaN,NaN,False,1.0


In [ ]:
set(ba_pps) - set(gba.participant_id)

{'00078',
 '00079',
 '00097',
 '00152',
 '00204',
 '00215',
 '00218',
 '00222',
 '00224',
 '00225',
 '00226',
 '00227',
 '00228',
 '00229',
 '00234',
 '00238',
 '00247',
 '00257',
 '00267',
 '00268',
 '00277',
 '00288',
 '00324',
 '00329',
 '00330',
 '00381',
 '00385',
 '00393',
 '00394',
 '00400',
 '00410',
 '00411',
 '00420',
 '00432',
 '00433',
 '00434',
 '00435',
 '00440',
 '00441',
 '00442',
 '00443',
 '00444',
 '00445',
 '00446',
 '00447',
 '00450',
 '00469',
 '00481',
 '00483',
 '00484',
 '00489',
 '00505',
 '00511',
 '00515',
 '00539',
 '00541',
 '00543',
 '00546',
 '00547',
 '00548',
 '00552',
 '00553',
 '00555',
 '00556',
 '00560',
 '00561',
 '00563',
 '00564',
 '00566',
 '00569',
 '00570',
 '00571',
 '00574',
 '00580',
 '00589',
 '00593',
 '00595',
 '00597',
 '00603',
 '00604',
 '00608',
 '00624',
 '00625',
 '00626',
 '00627',
 '00628',
 '00633',
 '00634',
 '00638',
 '00643',
 '00646',
 '00659',
 '00683',
 '00695',
 '00755',
 '00765',
 '00785',
 '00791',
 '00798',
 '00856',


### Getting standard drinks per day in last three months before baseline
Used variables: 
- An wie vielen Tagen tranken Sie in den letzten 3 Monaten zumindest 1 Glas Alkohol? (bx_qf_alc_01)
- Welches Format hatte die Angabe der Tage in der vorhergehenden Frage? (bx_qf_alc_02)
  - Angabe in den letzten 3 Monaten. (1)
  - Angabe pro Woche innerhalb der letzten 3 Monat. (2)
- An diesen Tagen an denen Sie in den letzten 3 Monaten Alkohol tranken: Können Sie mir anhand der Abbildungen auf der Liste angeben, was und wie viel Sie üblicherweise an einem Tag tranken?" (bx_qf1_sum)

In [ ]:
ba = dp.get_ba_data(columns = ['participant_id','redcap_event_name','mov_id','bx_qf_alc_01','bx_qf_alc_02','bx_qf1_sum']).query("redcap_event_name=='erhebungszeitpunkt_arm_1'")
ba.loc[(ba.participant_id=='11303') & (ba.bx_qf_alc_02==2),'bx_qf_alc_02'] = 1
ba.loc[(ba.participant_id=='11303')]

,participant_id,redcap_event_name,mov_id,bx_qf_alc_01,bx_qf_alc_02,bx_qf1_sum
13544,11303,erhebungszeitpunkt_arm_1,m190,24.0,1.0,3.2
13551,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13552,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13553,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13554,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13555,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13556,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13557,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13558,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN
13559,11303,erhebungszeitpunkt_arm_1,m190,None,NaN,NaN


In [ ]:
#export
@patch
def get_baseline_drinking_data(self:DataProvider):
    # Getting relevant data
    ba = self.get_ba_data(columns = ['participant_id','redcap_event_name','mov_id','bx_qf_alc_01','bx_qf_alc_02','bx_qf1_sum']).query("redcap_event_name=='erhebungszeitpunkt_arm_1'")
    # Correct one variable for one participant.  This participant reported drinking per three months but the data as logged as drinking per week
    ba.loc[(ba.participant_id=='11303') & (ba.bx_qf_alc_02==2),'bx_qf_alc_02'] = 1
    ba['drinking_days_last_three_month'] = ba['bx_qf_alc_01'].astype(float) * ba['bx_qf_alc_02'].replace({2:12})
    ba['drinks_per_drinking_day_last_three_month'] = ba['bx_qf1_sum']
    ba['drinks_per_day_last_three_month'] = (ba['drinking_days_last_three_month'] * ba['bx_qf1_sum'])/90
    standard_last_three = ba[~ba.drinks_per_day_last_three_month.isnull()][['mov_id','drinks_per_day_last_three_month','drinks_per_drinking_day_last_three_month','drinking_days_last_three_month']]
    standard_last_three.columns = ['participant','last_three_month','drinks_per_drinking_day_last_three_month','drinking_days_last_three_month']
    standard_last_three = standard_last_three.groupby('participant').first()
    return standard_last_three

In [ ]:
dp.get_baseline_drinking_data()

,last_three_month,drinks_per_drinking_day_last_three_month,drinking_days_last_three_month
participant,,,
b000,5.666667,8.50,60.0
b001,0.800000,2.00,36.0
b002,1.860000,4.65,36.0
b012,2.880000,5.40,48.0
b013,6.300000,6.75,84.0
...,...,...,...
m350,7.177778,17.00,38.0
m351,2.400000,3.00,72.0
m352,2.200000,5.50,36.0


### Getting Movisense data

In [ ]:
#export
@patch
def get_duplicate_mov_ids(self:DataProvider):
    '''This function creates a dictionary mapping old to new movisens IDs.'''
    df = self.get_ba_data()
    replace_dict_1 = dict(zip(df.mov_id_old, df.mov_id))
    replace_dict_2 = dict(zip(df.mov_id_old_2, df.mov_id))
    replace_dict = {**replace_dict_1, **replace_dict_2}
    try:
        del replace_dict[np.nan]
    except:
        pass
    if None in replace_dict.keys():
        del replace_dict[None]
    replace_dict['d033'] = 'd092' # This participant's data is currently missing in redcap, but they did change ID from 33 to 92
    return replace_dict

In [ ]:
#export
@patch
def get_old_mov_ids(self:DataProvider):
    old_ids = {}
    for old_id, new_id in self.get_duplicate_mov_ids().items():
        if new_id not in old_ids.keys():
            old_ids[new_id] = old_id
        else:
            old_ids[new_id] += ', '+old_id
    return old_ids

In [ ]:
assert len(dp.get_duplicate_mov_ids()), "Could not load duplicate_mov_ids"

In [ ]:
#export
@patch
@get_efficiently
def get_mov_data(self:DataProvider):
    """
    This function gets Movisense data
    1) We create unique participnat IDs (e.g. "b001"; this is necessary as sites use overapping IDs)
    2) We merge double IDs, so participants with two IDs only have one (for this duplicate_ids.csv has to be updated)
    3) We remove pilot participants
    4) We get starting dates (from the participant overviews in movisense; downloaded as html)
    5) We calculate sampling days and end dates based on the starting dates
    """
    # Loading raw data
    mov_berlin = pd.read_csv(self.mov_berlin_path, sep = ';')
    mov_dresden = pd.read_csv(self.mov_dresden_path, sep = ';')
    mov_mannheim = pd.read_csv(self.mov_mannheim_path, sep = ';')

    # Merging (participant numbers repeat so we add the first letter of location)
    mov_berlin['location'] = 'berlin'
    mov_dresden['location'] = 'dresden'
    mov_mannheim['location'] = 'mannheim'
    df = pd.concat([mov_berlin,mov_dresden,mov_mannheim])
    df['participant'] =  df['location'].str[0] + df.Participant.apply(lambda x: '%03d'%int(x))
    df.drop(columns = 'Participant', inplace = True) # Dropping old participant column to avoid mistakes
    df['trigger_date'] = pd.to_datetime(df.Trigger_date + ' ' + df.Trigger_time)

    # Merging double IDs (for participants with several movisense IDs)
    df['participant'] = df.participant.replace(self.get_duplicate_mov_ids())
    df['old_ids'] = df.participant.replace(self.get_old_mov_ids())

    # Removing pilot participants
    df = df[~df.participant.astype(str).str.contains('test')]
    df = df[~df.participant.isin(['m157', 'b010', 'b006', 'd001', 'd002', 'd042', 'm024', 'm028', 'm071', 'm079', 'm107'])]


    # Adding starting dates to get sampling days
    def get_starting_dates(path, pp_prefix = ''):
        soup = bs(open(path).read())
        ids = [int(x.text) for x in soup.find_all("td", class_ = 'simpleId')]
        c_dates = [x.find_all("span")[0]['title'] for x in soup.find_all("td", class_ = 'coupleDate')]
        s_dates = [x['value'] for x in soup.find_all("input", class_ = 'dp startDate')]
        df = pd.DataFrame({'participant':ids,'coupling_date':c_dates,'starting_date':s_dates})
        df['coupling_date'] = pd.to_datetime(df.coupling_date)
        df['starting_date'] = pd.to_datetime(df.starting_date)
        df.starting_date.fillna(df.coupling_date,inplace = True)
        df['participant'] = pp_prefix + df.participant.apply(lambda x: '%03d'%int(x))
        return df

    starting_dates = pd.concat([
    get_starting_dates(self.mov_berlin_starting_dates_path, 'b'),
    get_starting_dates(self.mov_dresden_starting_dates_path, 'd'),
    get_starting_dates(self.mov_mannheim_starting_dates_path, 'm')
    ])
    # For participants with several movisense IDs we use the first coupling date
    starting_dates.participant.replace(self.get_duplicate_mov_ids(), inplace = True)
    starting_dates = starting_dates.groupby('participant')[['starting_date','coupling_date']].min().reset_index()
    df = df.merge(starting_dates, on="participant", how = 'left', indicator = True)
    # Checking if starting dates were downloaded
    if "left_only" in df._merge.unique():
        no_starting_dates = df.query('_merge == "left_only"').participant.unique()
        print("Starting dates missing for participants below.  Did you download the participant overviews as html?", no_starting_dates)
    # Calculating movisense sampling day, adding date and end_date
    df['sampling_day'] = (df['trigger_date'] - df['starting_date']).dt.days + 1
    df['date'] = df.trigger_date.dt.date
    df['end_date'] = df.starting_date + pd.DateOffset(days = 365)
    df.index.rename('mov_index',inplace = True)
    # Adding redcap IDs
    ids_table = self.get_ba_data()[['participant_id','mov_id']].query('mov_id==mov_id').groupby('mov_id').first()
    ids_table.columns = ['redcap_id']
    df = df.merge(ids_table, left_on='participant', right_index = True, how = 'left')
    # Filtering out participants with no associated redcap data
    no_redcap = df.query("redcap_id.isna()", engine="python").participant.unique()
    print("Participants: %s have no associated redcap IDs and are excluded from the following analyses."%', '.join(no_redcap))
    df = df[df.redcap_id.isna()==False]
    return df

> Warning: Note that some participants do not have associated redcap data (see below).  These participants are excluded from further data analyses.

In [ ]:
mov_data = dp.get_mov_data(update=True)

Preparing mov_data
Participants: b186, b240, b295, b313, b317, b331, b353, b357, d161, d188, d207, m158, m257, m283, m304, m338 have no associated redcap IDs and are excluded from the following analyses.


In [ ]:
mov_data.query('participant=="d091"').old_ids

mov_index
171233    d070
171234    d070
171235    d070
171236    d070
171237    d070
          ... 
185365    d070
185366    d070
185367    d070
185368    d070
185369    d070
Name: old_ids, Length: 1216, dtype: object

In [ ]:
mov_data.query('participant=="m281"')

,Trigger,Trigger_date,Trigger_time,Trigger_counter,Form,Form_start_date,Form_start_time,Form_finish_date,Form_finish_time,Form_upload_date,...,participant,trigger_date,old_ids,starting_date,coupling_date,_merge,sampling_day,date,end_date,redcap_id
mov_index,,,,,,,,,,,,,,,,,,,,,


#### Structure of movisense data (selected columns)

In [ ]:
dp.get_mov_data()[['participant','starting_date','Trigger','Trigger_date','Trigger_counter','Form','INT_Coverage_kleinesBier','INT_Coverage_kleinerWeisswein']]

,participant,starting_date,Trigger,Trigger_date,Trigger_counter,Form,INT_Coverage_kleinesBier,INT_Coverage_kleinerWeisswein
mov_index,,,,,,,,
0,b001,2020-02-24,Button Pressed: Spiele starten und Initialfrag...,2020-02-24,9,Tag 1 Info,NaN,NaN
1,b001,2020-02-24,Fixed Time: 09:00,2020-02-24,11,Missing,NaN,NaN
2,b001,2020-02-24,Fixed Time: 12:00,2020-02-24,17,Coverage,NaN,NaN
3,b001,2020-02-24,Fixed Time: 12:00,2020-02-24,17,Craving,NaN,NaN
4,b001,2020-02-24,Fixed Time: 12:00,2020-02-24,17,MDBF,NaN,NaN
...,...,...,...,...,...,...,...,...
430375,m355,2021-11-15,Fixed Time: 21:00,2022-07-30,82,Missing,NaN,NaN
430376,m355,2021-11-15,Fixed Time: 12:00,2022-07-31,88,Missing,NaN,NaN
430377,m355,2021-11-15,Fixed Time: 14:00,2022-07-31,92,Missing,NaN,NaN


### Getting two-day data
This is a subset of the movisense data (only the two-daily questions), which we reformat so that each day is one one line.  We also add nan data for dates after the last data was received until 365 days until the starting date.  We rearrange answers to retrospective questions, so they are associated with the date in question rather than the date on which the question was answered.  For example, the answer to the question "How many beers did you drink two days ago" gets shifted two days back from the date of the question.

In [ ]:
#export
@patch
def get_alcohol_per_drink(self:DataProvider):
    if not os.path.isfile(self.alcohol_per_drink_path):
        return None
    else:
        return pd.read_csv(self.alcohol_per_drink_path)

In [ ]:
assert type(dp.get_alcohol_per_drink()) == type(pd.DataFrame()), "Could not load 'alcohol_per_drink.csv' from external data"

> We use this conversion table to convert participants' answers into ml_alc

In [ ]:
dp.get_alcohol_per_drink()[['name','amount','unit','vol','ml_alc_per_drink']]

,name,amount,unit,vol,ml_alc_per_drink
0,Kleines Bier,0.20,Liter,5.0,10.00
1,Mittleres Bier,0.33,Liter,5.0,16.50
2,Großes Bier,0.50,Liter,5.0,25.00
3,Kleiner Weißwein,0.10,Liter,11.0,11.00
4,Mittlerer Weißwein,0.20,Liter,11.0,22.00
5,Flasche Weißwein,0.75,Liter,11.0,82.50
6,Kleiner Rotwein,0.10,Liter,12.0,12.00
7,Mittlerer Rotwein,0.20,Liter,12.0,24.00
8,Flasche Rotwein,0.75,Liter,12.0,90.00
9,Sekt,0.10,Liter,11.5,11.50


In [ ]:
#export
@patch
@get_efficiently
def get_two_day_data(self:DataProvider):
    """Gets two-day which are a subset of mov_data
    1) We select two-day forms and turn the dataframe into one row per participant-date (Form repetitions on the same day are removed)
    2) We reduce all drinking questions to an easily readable dictionary of drink amounts.
    3) We add missing data for dates on which we did not receive answers (starting 2 days before starting date up to 365 days after)
    4) We shift answers from retrospective drinking questions backwards to the adequate date (e.g. one day back for yesterday questions)
    """
    mov_data = self.get_mov_data(update = True).reset_index()
    drinking_columns = [c for c in mov_data.columns if c.startswith("Anzahl") and "10day" not in c]
    mdbf_columns = [c for c in mov_data.columns if "MDBF" in c and "INT" not in c]
    two_day_columns = ['mov_index','starting_date','end_date','sampling_day'] + drinking_columns + mdbf_columns + ['soziale_isolation','alternative_rewards','craving','Limit','Kontrolle','Form_upload_date']
    # 1) Turning df into one line per date
    two_day_forms = ["Coverage","Soziale Isolation","Craving","MDBF","Alternative Rewards","Limits & Control"]
    df = mov_data.sort_values(by=['participant','trigger_date','Form','Trigger_counter'])
    df = df[df.Form.isin(two_day_forms)].groupby(["participant","date"])[two_day_columns].first().dropna(how='all').reset_index()
    # 2) Turning drinking answers into dictionaries
    df[drinking_columns].fillna(0,inplace = True)
    df['drinks_gestern'] = df[[c for c in drinking_columns if "vorgestern" not in c]].fillna(0).agg(lambda x: x.to_dict(), axis=1)
    df['drinks_vorgestern'] = df[[c for c in drinking_columns if "vorgestern" in c]].fillna(0).agg(lambda x: x.to_dict(), axis=1)
    # Adding missing values
    def add_missing_data(df):
        dates = pd.date_range(df.starting_date.iloc[0]-pd.DateOffset(days = 2), df.end_date.iloc[0])
        df = df.reindex(dates)
        df.index.names = ['date']
        df[['starting_date','sampling_day']] = df[['starting_date','sampling_day']].bfill().ffill()
        # 3) Shifting back retrospective answers
        df['drinks'] = df['drinks_gestern'].shift(-1) # Drinks yesterday get shifted back one day
        df['drinks'].fillna(df['drinks_vorgestern'].shift(-2), inplace = True) # Drinks before yesterday shift back two days
        df['limit'] = df.Limit.ffill(limit=8) # Limit gets repeated forward for eight days
        df['control'] = df.Kontrolle.bfill(limit=8) # Control is repeated backward for eight days
        df['upload'] = df.Form_upload_date.shift(-1)
        df['upload'] = df.upload.bfill(limit=1)
        return df
    df = df.set_index('date').groupby('participant').apply(add_missing_data).drop(columns='participant').reset_index()


    # 4) Calculating daily alcohol consumption in grams
    alc_dict = {"Anzahl"+k:v for k,v in self.get_alcohol_per_drink().set_index('drink')['ml_alc_per_drink'].to_dict().items()}
    def calculate_g_alc(x):
        if x != x:
            ml_alc = np.nan
        else:
            ml_alc = 0
            for k,v in x.items():
                if v > 0:
                    ml_alc += alc_dict[k.split('_')[0]] * v
        return ml_alc * .8
    df['g_alc']  = df.drinks.apply(calculate_g_alc)
    # Returning relevant columns only
    df = df.reset_index()
    df['sampling_day'] = df.groupby('participant').starting_date.cumcount() + 1
    df.index.rename('two_day_index',inplace = True)
    return df[['mov_index','participant','starting_date','date','sampling_day'] + mdbf_columns + ['soziale_isolation','alternative_rewards','craving','Limit','limit','Kontrolle','control','drinks','g_alc','drinks_vorgestern','drinks_gestern','upload']]

In [ ]:
def test_one_line_per_participant_day():
    return dp.get_two_day_data().groupby(['participant','date']).date.count().value_counts().index[0] == 1

assert test_one_line_per_participant_day(), "Two day data does not have one line per participant day."

#### Structure of two-day data
> Warning: Note that sampling days run further than 365.  This should not be the case.  It turns out that for these participants the end date was manually set further than one year after the start date, on the movisens page.

#### Drinking inconsistencies

##### Raw data

In [ ]:
raw_mov = pd.read_csv('/Users/hilmarzech/Projects/trr265/trr265/data/raw/mov_data_m.csv', sep = ';')

On the 10th they drank one large beer.

In [ ]:
raw_mov.query('(Participant==13)').Trigger_date.max()

'2021-04-03'

In [ ]:
raw_mov.query('(Participant==13) & (Form=="Coverage")').sort_values(by='Trigger_date').set_index('Trigger_date').filter(like='Anzahl').dropna(how = 'all').index

Index(['2020-07-08', '2020-07-12', '2020-07-14', '2020-07-16', '2020-07-26',
       '2020-08-01', '2020-08-03', '2020-08-07', '2020-08-17', '2020-08-23',
       '2020-08-25', '2020-08-25', '2020-08-26', '2020-08-26', '2020-08-26',
       '2020-08-26', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-28',
       '2020-08-28', '2020-08-29', '2020-08-30', '2020-08-30', '2020-09-02',
       '2020-09-03', '2020-09-04', '2020-09-06', '2020-09-06', '2020-09-08',
       '2020-09-10', '2020-09-12', '2020-09-14', '2020-09-14', '2020-09-16',
       '2020-09-18', '2020-09-21', '2020-09-23', '2020-09-25', '2020-09-27',
       '2020-09-27', '2020-09-29', '2020-10-01', '2020-10-01', '2020-10-03',
       '2020-10-03', '2020-10-05', '2020-10-07', '2020-10-09', '2020-10-09',
       '2020-10-11', '2020-10-11', '2020-10-15', '2020-10-17', '2020-10-19',
       '2020-10-21', '2020-10-27', '2020-10-29', '2020-10-31', '2020-11-02',
       '2020-11-10', '2020-11-12', '2020-11-16', '2020-11-18', '2020-11-22',

In [ ]:
raw_mov.query('(Participant==13) & (Form=="Coverage")').sort_values(by='Trigger_date').set_index('Trigger_date').filter(like='Bier').iloc[90:110]

,Anzahl_kleinesBier_10day,Anzahl_mittleresBier_10day,Anzahl_grosseBier_10day,AnzahlKleinesBier,AnzahlMittlereBier,AnzahlGrosseBier,AnzahlKleinesBier_vorgestern,AnzahlMittlereBier_vorgestern,AnzahlGrosseBier_vorgestern,Anzahl2_kleinesBier_10day,Anzahl2_mittleresBier_10day,Anzahl2_grossesBier_10day
Trigger_date,,,,,,,,,,,,
2020-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On the 11th, the participant drank one large beer.

In [ ]:
raw_mov.query('(Participant==125) & (Form=="Coverage") & (Trigger_date=="2021-02-24")').Form_upload_date

58039    2021-02-24
Name: Form_upload_date, dtype: object

In [ ]:
raw_mov.filter(like='upload')

,Form_upload_date,Form_upload_time
0,2020-02-24,15:32:40
1,2020-02-24,15:32:40
2,2020-02-24,15:32:41
3,2020-02-24,15:32:41
4,2020-02-24,15:32:41
...,...,...
129693,2022-08-01,12:37:13
129694,2022-08-01,12:37:14
129695,2022-08-01,12:37:14
129696,2022-08-01,12:37:15


In [ ]:
two_day = dp.get_two_day_data(update = True)
two_day['date'] = two_day.date.dt.strftime("%Y-%m-%d")

Preparing two_day_data


In [ ]:
nans

,mov_index,participant_x,starting_date_x,date,sampling_day_x,redcap_id,MDBF_zufrieden_x,MDBF_ruhig_x,MDBF_wohl_x,MDBF_entspannt_x,...,intention_mean_1,social_double_mean,ln_alc,binge,VAR00001,diff_date,VAR00002,alc_compare,limit_compare,same_nan
0,NaN,b001,2020-02-24,2020-10-01,223.0,10012,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1.0
1,NaN,b001,2020-02-24,2020-10-02,224.0,10012,NaN,NaN,NaN,NaN,...,2.000000,1.989130,NaN,NaN,2020-12-16,-75.0,-75.0,NaN,0.0,1.0
2,NaN,b001,2020-02-24,2020-10-03,225.0,10012,NaN,NaN,NaN,NaN,...,2.000000,1.989130,NaN,NaN,2020-12-16,-74.0,-74.0,NaN,0.0,1.0
3,NaN,b001,2020-02-24,2020-10-04,226.0,10012,NaN,NaN,NaN,NaN,...,2.000000,1.989130,0.000000,0.0,2020-12-16,-73.0,-73.0,0.0,0.0,1.0
4,NaN,b001,2020-02-24,2020-10-05,227.0,10012,NaN,NaN,NaN,NaN,...,2.000000,1.989130,2.197225,0.0,2020-12-16,-72.0,-72.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21143,303014.0,m290,2021-02-08,2021-02-25,20.0,11008,3.0,3.0,3.0,2.0,...,2.400000,1.100000,0.000000,0.0,2020-12-16,71.0,71.0,0.0,0.0,1.0
21144,NaN,m290,2021-02-08,2021-02-26,21.0,11008,NaN,NaN,NaN,NaN,...,2.400000,1.100000,4.609959,1.0,2020-12-16,72.0,72.0,0.0,0.0,1.0
21145,303019.0,m290,2021-02-08,2021-02-27,22.0,11008,1.0,6.0,7.0,2.0,...,2.400000,1.100000,0.000000,0.0,2020-12-16,73.0,73.0,0.0,0.0,1.0
21146,NaN,m290,2021-02-08,2021-02-28,23.0,11008,NaN,NaN,NaN,NaN,...,2.400000,1.100000,0.000000,0.0,2020-12-16,74.0,74.0,0.0,0.0,1.0


In [ ]:
nans = pd.read_spss('/Users/hilmarzech/Desktop/sparse_old_new0308.sav')

In [ ]:
unequal = nans[nans['g_alc_x'].isna() != nans['g_alc_y'].isna()]#.participant_x.unique()
unequal = unequal[unequal.participant_x.isin(['m065', 'm069'])==False]


In [ ]:
unequal['date'] = unequal.date.dt.strftime("%Y-%m-%d")

In [ ]:
unequal['participant'] = unequal['participant_x']


In [ ]:
merged = unequal.merge(two_day,on = ['participant','date'], indicator = 'indicate')

In [ ]:
pd.to_datetime(merged.upload).describe()

count                     142
unique                     19
top       2021-07-28 00:00:00
freq                       74
first     2020-10-02 00:00:00
last      2021-07-28 00:00:00
Name: upload, dtype: object

In [ ]:
merged.upload.value_counts().sort_index()

2020-10-02    17
2020-10-03    23
2020-10-04     1
2020-10-15     1
2021-02-11     2
2021-02-13     2
2021-02-15     2
2021-02-17     2
2021-02-19     2
2021-02-22     2
2021-02-23     2
2021-02-24     2
2021-02-25     2
2021-02-26     2
2021-02-27     2
2021-02-28     2
2021-03-02     1
2021-03-11     1
2021-07-28    74
Name: upload, dtype: int64

In [ ]:
merged.Form_upload_date.values

array(['2020-10-01', '2021-03-11', '2021-03-11', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2021-07-28',
       '2021-07-28', '2021-07-28', '2021-07-28', '2020-10-01',
       '2020-10-01', '2020-10-01', '2020-10-01', '2020-10-01',
       '2020-10-01', '2020-10-01', '2020-10-01', '2020-10-01',
       '2020-10-01', '2020-10-01', '2021-02-22', '2021-02-24',
       '2021-02-26', '2021-02-28', '2020-10-01', '2020-10-01',
       '2020-10-01', '2020-10-01', '2020-10-03', '2020-10-01',
       '2020-10-01', '2020-10-01', '2020-10-01', '2020-10-15',
       '2021-02-11', '2021-02-13', '2021-02-15', '2021-02-17',
       '2021-02-23', '2021-02-25'], dtype=object)

In [ ]:
unequal['participant'] = unequal['participant_x']
unequal = unequal.set_index(['participant','date'])

In [ ]:
unequal

mov_index participant_x starting_date_x  \
participant date                                                  
b012        2020-10-01        NaN          b012      2020-06-28   
b013        2020-10-01     3564.0          b013      2020-06-28   
            2020-11-18     3911.0          b013      2020-06-28   
            2020-11-19        NaN          b013      2020-06-28   
            2020-11-20     3916.0          b013      2020-06-28   
...                           ...           ...             ...   
m143        2021-02-22        NaN          m143      2020-11-30   
            2021-02-23   304597.0          m143      2020-11-30   
            2021-02-24        NaN          m143      2020-11-30   
            2021-02-25   304604.0          m143      2020-11-30   
            2021-02-26        NaN          m143      2020-11-30   

                        sampling_day_x redcap_id  MDBF_zufrieden_x  \
participant date                                                     
b012        2020-10-01            98.0     10077               NaN   
b013        2020-10-01            98.0     00149               2.0   
            2020-11-18           146.0     00149               2.0   
            2020-11-19           147.0     00149               NaN   
            2020-11-20           148.0     00149               2.0   
...                                ...       ...               ...   
m143        2021-02-22            87.0     10689               NaN   
            2021-02-23            88.0     10689               2.0   
            2021-02-24            89.0     10689               NaN   
            2021-02-25            90.0     10689               3.0   
            2021-02-26            91.0     10689               NaN   

                        MDBF_ruhig_x  MDBF_wohl_x  MDBF_entspannt_x  \
participant date                                                      
b012        2020-10-01           NaN          NaN               NaN   
b013        2020-10-01           7.0          7.0               2.0   
            2020-11-18           6.0          6.0               2.0   
            2020-11-19           NaN          NaN               NaN   
            2020-11-20           6.0          6.0               2.0   
...                              ...          ...               ...   
m143        2021-02-22           NaN          NaN               NaN   
            2021-02-23           4.0          7.0               4.0   
            2021-02-24           NaN          NaN               NaN   
            2021-02-25           5.0          5.0               2.0   
            2021-02-26           NaN          NaN               NaN   

                        soziale_isolation_x  ...  intention_mean_1  \
participant date                             ...                     
b012        2020-10-01                  NaN  ...               NaN   
b013        2020-10-01                  1.0  ...               NaN   
            2020-11-18                  1.0  ...          3.000000   
            2020-11-19                  NaN  ...          3.000000   
            2020-11-20                  1.0  ...          3.000000   
...                                     ...  ...               ...   
m143        2021-02-22                  NaN  ...          2.538462   
            2021-02-23                  1.0  ...          2.538462   
            2021-02-24                  NaN  ...          2.538462   
            2021-02-25                  1.0  ...          2.538462   
            2021-02-26                  NaN  ...          2.538462   

                        social_double_mean  ln_alc  binge   VAR00001  \
participant date                                                       
b012        2020-10-01                 NaN     NaN    NaN        NaT   
b013        2020-10-01                 NaN     NaN    NaN        NaT   
            2020-11-18                 1.0     NaN    NaN 2020-12-16   
            2020-11-19                 1.0    

In [ ]:
mov_data['date'] = pd.to_datetime(mov_data.date)

In [ ]:
mov_data.filter(like='upload')

,Form_upload_date,Form_upload_time
mov_index,,
0,2020-02-24,15:32:40
1,2020-02-24,15:32:40
2,2020-02-24,15:32:41
3,2020-02-24,15:32:41
4,2020-02-24,15:32:41
...,...,...
430375,2022-08-01,13:30:21
430376,2022-08-01,13:30:26
430377,2022-08-01,13:30:24


In [ ]:
mov_data.participant

mov_index
0         b001
1         b001
2         b001
3         b001
4         b001
          ... 
430375    m355
430376    m355
430377    m355
430378    m355
430379    m355
Name: participant, Length: 427179, dtype: object

In [ ]:
unequal

AttributeError: 'DataFrame' object has no attribute 'date'

In [ ]:
unequal.reset_index().date.values

array(['2020-10-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2020-11-18T00:00:00.000000000', '2020-11-19T00:00:00.000000000',
       '2020-11-20T00:00:00.000000000', '2020-11-21T00:00:00.000000000',
       '2020-11-22T00:00:00.000000000', '2020-11-23T00:00:00.000000000',
       '2020-11-24T00:00:00.000000000', '2020-11-25T00:00:00.000000000',
       '2020-12-01T00:00:00.000000000', '2020-12-02T00:00:00.000000000',
       '2020-12-03T00:00:00.000000000', '2020-12-04T00:00:00.000000000',
       '2020-12-07T00:00:00.000000000', '2020-12-08T00:00:00.000000000',
       '2020-12-09T00:00:00.000000000', '2020-12-10T00:00:00.000000000',
       '2020-12-11T00:00:00.000000000', '2020-12-12T00:00:00.000000000',
       '2020-12-13T00:00:00.000000000', '2020-12-14T00:00:00.000000000',
       '2020-12-15T00:00:00.000000000', '2020-12-16T00:00:00.000000000',
       '2020-12-17T00:00:00.000000000', '2020-12-18T00:00:00.000000000',
       '2020-12-19T00:00:00.000000000', '2020-12-20

In [ ]:
unequal['upload'] = mov_data.groupby(['participant','date']).first()['Form_upload_date']

In [ ]:
unequal.upload

participant  date      
b012         2020-10-01           NaN
b013         2020-10-01    2020-10-01
             2020-11-18    2021-03-11
             2020-11-19           NaN
             2020-11-20    2021-07-28
                              ...    
m143         2021-02-22           NaN
             2021-02-23    2021-02-23
             2021-02-24           NaN
             2021-02-25    2021-02-25
             2021-02-26           NaN
Name: upload, Length: 142, dtype: object

In [ ]:
unequal.merge(mov_data.query('Form=="Coverage"'), left_on=[['participant_x','date']], right_on=[['participant','date']])

KeyError: ['participant', 'date']

In [ ]:
nans[['g_alc_x','g_alc_y']]

KeyError: "None of [Index(['g_alc_x', 'g_alc_y'], dtype='object')] are in the [columns]"

In [ ]:
raw_mov.query('(Participant==69) & (Form=="Coverage") & (Trigger_date == "2020-10-12")').filter(like='_gestern_').iloc[0]

Alkoholische_Getraenke_gestern_1     0.0
Alkoholische_Getraenke_gestern_2     0.0
Alkoholische_Getraenke_gestern_3     1.0
Alkoholische_Getraenke_gestern_4     0.0
Alkoholische_Getraenke_gestern_5     0.0
Alkoholische_Getraenke_gestern_6     0.0
Alkoholische_Getraenke_gestern_7     0.0
Alkoholische_Getraenke_gestern_8     0.0
Alkoholische_Getraenke_gestern_9     0.0
Alkoholische_Getraenke_gestern_10    0.0
Alkoholische_Getraenke_gestern_11    0.0
Alkoholische_Getraenke_gestern_12    0.0
Alkoholische_Getraenke_gestern_13    0.0
Alkoholische_Getraenke_gestern_14    0.0
Alkoholische_Getraenke_gestern_15    0.0
Alkoholische_Getraenke_gestern_16    0.0
Alkoholische_Getraenke_gestern_17    0.0
Alkoholische_Getraenke_gestern_18    0.0
Alkoholische_Getraenke_gestern_19    0.0
Alkoholische_Getraenke_gestern_20    0.0
Alkoholische_Getraenke_gestern_21    0.0
Name: 44836, dtype: float64

##### Mov_data

In [ ]:
mov_data = dp.get_mov_data()

on the 10th one large beer

the participant drank a large red wine 

In [ ]:
with pd.option_context('display.max_rows', None,):
    display(mov_data.query('(participant=="b013") & (Form=="Coverage")').query('Trigger_date=="2020-12-25"').filter(like='Anzahl').T.loc['AnzahlFlaschenRotwein_vorgestern'])

mov_index
4025    1.0
Name: AnzahlFlaschenRotwein_vorgestern, dtype: float64

In [ ]:
mov_data.Form_upload_date

mov_index
0         2020-02-24
1         2020-02-24
2         2020-02-24
3         2020-02-24
4         2020-02-24
             ...    
430375    2022-08-01
430376    2022-08-01
430377    2022-08-01
430378    2022-08-01
430379    2022-08-03
Name: Form_upload_date, Length: 427179, dtype: object

In [ ]:
with pd.option_context('display.max_rows', None,):
    display(two_day.query('(participant=="b013")').query('date==20201225').filter(like='drinks_vorgestern').iloc[0].iloc[0])

{'AnzahlKleinesBier_vorgestern': 0.0,
 'AnzahlMittlereBier_vorgestern': 0.0,
 'AnzahlGrosseBier_vorgestern': 0.0,
 'AnzahlKleinerWeisswein_vorgestern': 0.0,
 'AnzahlMittlererWeisswein_vorgestern': 0.0,
 'AnzahlFlascheWeisswein_vorgestern': 0.0,
 'AnzahlKleinerRotwein_vorgestern': 0.0,
 'AnzahlMittlererRotwein_vorgestern': 0.0,
 'AnzahlFlaschenRotwein_vorgestern': 1.0,
 'AnzahlSekt_vorgestern': 0.0,
 'AnzahlFlaschenSekt_vorgestern': 0.0,
 'AnzahlLikoerWein_vorgestern': 0.0,
 'AnzahlKleineLikoer_vorgestern': 0.0,
 'AnzahlGrosseLikoer_vorgestern': 0.0,
 'AnzahlLikoerSuess_vorgestern': 0.0,
 'AnzahlKleineSpirituosen_vorgestern': 0.0,
 'AnzahlGrosseSpirituosen_vorgestern': 0.0,
 'AnzahlSpirituosen100ml_vorgestern': 0.0,
 'AnzahlSpirituosenStark_vorgestern': 0.0}

On the 11th one large beer

##### Two day data

In [ ]:
two_day = dp.get_two_day_data(update = True)

Preparing two_day_data


On the 10th, 1 large beer.

In [ ]:
two_day.query('(participant=="b013") & (date == 20201223)')['g_alc']

two_day_index
1284    72.0
Name: g_alc, dtype: float64

On the 11th 3 large beers.

In [ ]:
dp.get_two_day_data()[['mov_index','participant','date','starting_date','sampling_day','soziale_isolation','drinks','g_alc']]#.iloc[:20]

Preparing two_day_data


,mov_index,participant,date,starting_date,sampling_day,soziale_isolation,drinks,g_alc
two_day_index,,,,,,,,
0,NaN,b001,2020-02-22,2020-02-24,1,NaN,"{'AnzahlKleinesBier_vorgestern': 0.0, 'AnzahlM...",6.4
1,NaN,b001,2020-02-23,2020-02-24,2,NaN,"{'AnzahlKleinesBier': 0.0, 'AnzahlMittlereBier...",35.2
2,6.0,b001,2020-02-24,2020-02-24,3,1.0,NaN,NaN
3,NaN,b001,2020-02-25,2020-02-24,4,NaN,NaN,NaN
4,NaN,b001,2020-02-26,2020-02-24,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
218955,NaN,m355,2022-11-11,2021-11-15,364,NaN,NaN,NaN
218956,NaN,m355,2022-11-12,2021-11-15,365,NaN,NaN,NaN
218957,NaN,m355,2022-11-13,2021-11-15,366,NaN,NaN,NaN
